In [1]:
import time
import pickle
import random

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
from tqdm.auto import tqdm
from fake_useragent import UserAgent

In [28]:
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')

# ua = UserAgent()
driver = webdriver.Chrome(options=options)
# driver.implicitly_wait(15)

In [45]:
UserAgent().random

'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1309.0 Safari/537.17'

In [29]:
with open("animated_film_listing.pickle", "rb") as file:
    animated_film_listing = pickle.load(file)

In [30]:
driver.get("https://movie.douban.com/subject/1291585/")

In [19]:
driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": ua.random})

{}

In [38]:
comments_section = driver.find_element_by_id("hot-comments")
for item in comments_section.find_elements_by_class_name("comment-item"):
    print(item.find_element_by_xpath("div[@class=\"comment\"]/h3/span[@class=\"comment-info\"]/a").text)

亂馬2/1
蕊蕊
有心打扰
同志亦凡人中文站
爱幻想的小孩


In [22]:
comments_section.find_elements_by_class_name("comment-item")[0].find_element_by_xpath("//span[@class=\"comment-info\"]/a").text

'亂馬2/1'

In [27]:
driver.close()

In [ ]:
start_idx = 0
end_idx = 2000

In [ ]:
for film in tqdm(animated_film_listing[start_idx:end_idx]):
    driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": ua.random})
    driver.get(film['page_link'])
    
    # fetch all info in plain text
    try:
        film_info = driver.find_element_by_id("info")
    except NoSuchElementException:
        print(driver.current_url)
        break
    info_dict = dict([line.split(": ", 1) for line in film_info.text.split("\n")])
    film.update(info_dict)
    film['全名'] = driver.find_element_by_css_selector("span[property='v:itemreviewed']").text

    try:
        film['年份'] = driver.find_element_by_class_name("year").text
        film['剧情简介'] = driver.find_element_by_css_selector("span[property='v:summary']").text
    except NoSuchElementException:
        pass
    
    
    # expand actors list
    try:
        film_info.find_element_by_class_name("more-actor").click()
    except:
        pass
    
    casts = []
    for actor in film_info.find_elements_by_css_selector("a[rel='v:starring']")[:10]:
        name = actor.text
        page_link = actor.get_attribute("href")
        casts.append({'name': name, 'page_link': page_link})
        
    film['主演'] = casts
    
    reviews = []
    try:
        comments_section = driver.find_element_by_id("hot-comments")
        for item in comments_section.find_elements_by_class_name("comment-item"):
            review = {}
            review['id'] = item.get_attribute("data-cid")
            review['user'] = item.find_element_by_xpath("div[@class=\"comment\"]/h3/span[@class=\"comment-info\"]/a").text
            review['time'] = item.find_element_by_class_name("comment-time").text
            review['text'] = item.find_element_by_class_name("short").text
            
            reviews.append(review)
    except NoSuchElementException:
        pass
        
    film['影评'] = reviews
    driver.delete_all_cookies()

In [ ]:
with open("animated_film_info_{}-{}.pickle".format(start_idx, end_idx-1), "wb") as file:
    pickle.dump(animated_film_listing[start_idx:end_idx], file)